In [1]:
# 基础的asm文件，用于构建CFG
fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/staticAnalysis/hw-memory/inputs/spce_changes/small/small.asm"

# 带bss段和data段的asm文件，用于查看全局变量地址空间
seg_fp = r'/Users/gugujixiao/workspace/project/HWMemory/Code/staticAnalysis/hw-memory/inputs/spce_changes/small/smallD.asm'

# 带源码的asm文件，用于源码和汇编代码对照
code_fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/staticAnalysis/hw-memory/inputs/spce_changes/small/smallL.asm"


In [2]:
'''读指令, asm文件对象化'''
from cfg.cfg import proc_identify
from cfg.isa import Instruction, AddrMode, InstructionType
from typing import Tuple
from cfg.read_asm import AsmFileReader, StatementType
from cfg.read_segment import segmentReader
from cfg.read_code import AsmCodeReader, MixStatementType

reader =AsmFileReader(fp)#asm对象
segreader = segmentReader(seg_fp)#带bss和data段的asm对象
codereader = AsmCodeReader(code_fp)#带源码对照的asm对象

statements = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)


for i in statements:
    if i[0] == StatementType.Instruction:
        ins = i[1]
        if ins.ins_type == InstructionType.LoadStore:
            print(ins.tokens,ins.is_nsp)

procs = proc_identify(statements)

# for proc in procs:
#     print(proc.name, proc.beg_addr.hex_str())
#     if False and proc.name == 'main':
#         for inst in proc.instruction:
#             print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
# print()

('40050c', 'a9bf7bfd', 'stp', None, 'x29, x30, [sp, #-16]!') False
('400518', 'a8c17bfd', 'ldp', None, 'x29, x30, [sp], #16') False
('400520', 'a9bf7bf0', 'stp', None, 'x16, x30, [sp, #-16]!') False
('400528', 'f947fe11', 'ldr', None, 'x17, [x16, #4088]') True
('400544', 'f9400211', 'ldr', None, 'x17, [x16]') True
('400554', 'f9400611', 'ldr', None, 'x17, [x16, #8]') True
('400564', 'f9400a11', 'ldr', None, 'x17, [x16, #16]') True
('400574', 'f9400e11', 'ldr', None, 'x17, [x16, #24]') True
('400584', 'f9401211', 'ldr', None, 'x17, [x16, #32]') True
('4005d0', 'f94003e1', 'ldr', None, 'x1, [sp]') False
('400608', 'f947ec00', 'ldr', None, 'x0, [x0, #4056]') True
('40063c', 'f947e821', 'ldr', None, 'x1, [x1, #4048]') True
('400678', 'f947f042', 'ldr', None, 'x2, [x2, #4064]') True
('400690', 'a9be7bfd', 'stp', None, 'x29, x30, [sp, #-32]!') False
('400698', 'f9000bf3', 'str', None, 'x19, [sp, #16]') True
('4006a0', '3940e260', 'ldrb', None, 'w0, [x19, #56]') True
('4006b0', '3900e260', 's

In [3]:
'''CFG:分proc'''
from graphviz import Digraph
from cfg.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
'''CFG:处理call_graph'''
from cfg.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output/call_graph.svg'

In [5]:
'''CFG:生成TCFG和loop信息'''
from cfg.cfg import TCfg


tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()


for l in tcfg.loops:
    print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    l.back_edge.is_backEdge = True
for n in tcfg.all_nodes:
     temp = n.inside_loop.name if n.inside_loop is not None else 'None'
     print("{} {}".format(n.name, temp))



#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




l2 (in l1 children []): n104->n105 ['n105', 'n104']
l1 (in l0 children []): n106->n107 ['n103', 'n104', 'n105', 'n106', 'n107']
l6 (in l3 children []): n93->n94 ['n93', 'n94']
l12 (in l11 children []): n83->n84 ['n84', 'n83']
l11 (in l7 children []): n85->n86 ['n82', 'n83', 'n84', 'n85', 'n86']
l19 (in l13 children []): n74->n75 ['n74', 'n75']
l28 (in l27 children []): n65->n66 ['n66', 'n65']
l27 (in l20 children []): n67->n68 ['n66', 'n65', 'n64', 'n68', 'n67']
l38 (in l29 children []): n56->n57 ['n57', 'n56']
l49 (in l39 children []): n46->n47 ['n44', 'n47', 'n45', 'n43', 'n46']
l61 (in l50 children []): n35->n36 ['n35', 'n34', 'n33', 'n32', 'n113', 'n36']
l74 (in l62 children []): n25->n26 ['n25', 'n24', 'n22', 'n23', 'n26', 'n112']
l0 (in None children []): n108->n109 ['n103', 'n104', 'n105', 'n106', 'n107', 'n108', 'n109', 'n102']
l3 (in None children []): n95->n96 ['n93', 'n92', 'n94', 'n95', 'n96']
l7 (in None children []): n87->n88 ['n87', 'n81', 'n88', 'n82', 'n83', 'n84', 'n8

In [6]:
'''生成插桩相关信息'''
# 生成loop中各node首尾地址
loop_range =list()
loop_range_hex =list()

with open("./output/loop_addr.txt","w") as f:
    for l in tcfg.loops:
        #f.write(l.name+"\n")
        #print(l.name)
        for n in l.all_nodes:
            ins = n.instructions
            #head = ins[0].addr
            head,end = n.inst_range
            string = l.name[1:]+" "+head.hex_str()+"-"+end.hex_str()+"\n"
            loop_range.append(([head.val(),end.val()],n.name,l.name))
            loop_range_hex.append(([head.hex_str(),end.hex_str()],n.name,l.name))
            f.write(string)
        f.write("\n")

# 生成源代码对于loop的对应关系
code_stat = list()# 中间变量
loop_code_range =list()# 最终输出


for i in range(len(codereader.statements)):
    s: Tuple[MixStatementType, tuple]
    if codereader.statements[i][0] == MixStatementType.codeline:
        codeline = codereader.statements[i][1]
        addr = ""
        not_find = True
        k = i
        while not_find:
            k += 1
            if codereader.statements[k][0] == MixStatementType.asm:
                addr =  int(codereader.statements[k][1][0],16) 
                not_find = False

        code_stat.append((addr,codeline,"loopname"))


with open("./output/loop_code.txt","w") as f:
    for i in code_stat:
        for l in loop_range:
            if l[0][0]<i[0]<l[0][1]:
                string = i[1]+l[1]
                loop_code_range.append((i[1],l[1]))
                f.write(string)
                f.write("\n")  


for i in loop_range_hex:
    print(i)

(['400d28', '400d30'], 'n105', 'l2')
(['400bf4', '400d24'], 'n104', 'l2')
(['400bec', '400bf0'], 'n103', 'l1')
(['400bf4', '400d24'], 'n104', 'l1')
(['400d28', '400d30'], 'n105', 'l1')
(['400d34', '400d3c'], 'n106', 'l1')
(['400d40', '400d48'], 'n107', 'l1')
(['400d84', '400e54'], 'n93', 'l6')
(['400e58', '400e60'], 'n94', 'l6')
(['400f78', '400f80'], 'n84', 'l12')
(['400ea4', '400f74'], 'n83', 'l12')
(['400e9c', '400ea0'], 'n82', 'l11')
(['400ea4', '400f74'], 'n83', 'l11')
(['400f78', '400f80'], 'n84', 'l11')
(['400f84', '400f8c'], 'n85', 'l11')
(['400f90', '400f98'], 'n86', 'l11')
(['400d84', '400e54'], 'n74', 'l19')
(['400e58', '400e60'], 'n75', 'l19')
(['400f78', '400f80'], 'n66', 'l28')
(['400ea4', '400f74'], 'n65', 'l28')
(['400f78', '400f80'], 'n66', 'l27')
(['400ea4', '400f74'], 'n65', 'l27')
(['400e9c', '400ea0'], 'n64', 'l27')
(['400f90', '400f98'], 'n68', 'l27')
(['400f84', '400f8c'], 'n67', 'l27')
(['400e58', '400e60'], 'n57', 'l38')
(['400d84', '400e54'], 'n56', 'l38')
(['

In [7]:
from rw_analysis.rw_core import loadstore_Obj
lds_obj = loadstore_Obj(segreader,tcfg)

#用于给后面解决兼容性问题的数据
lsproc = lds_obj.lsproc


""" ls_loop_info = lds_obj.loop_info
lds_table = lds_obj.lsproc.ls_table

for i in lds_table:
    if i.target_num == 1:
        print("single",i.ins.tokens,"最终地址",i.final_addr,i.addr_offset,"是否找到",i.is_find,i.node.name) 
    else:
        print("muti",i.ins.tokens,"最终地址",i.final_addr,i.addr_offset,"是否找到",i.is_find,i.node.name)


for k,v in ls_loop_info.items():
    print(k,v) """

('400db0', 'b8607840', 'ldr', None, 'w0, [x2, x0, lsl #2]') n56
('400dac', '8b030000', 'add', None, 'x0, x0, x3')
x2 False
x0 False
x3 False

('400da8', 'd37ef400', 'lsl', None, 'x0, x0, #2')
x2 False
x0 False
x3 False

('400da4', '8b010000', 'add', None, 'x0, x0, x1')
x2 False
x0 False
x3 False
x1 False

('400da0', 'd37ef400', 'lsl', None, 'x0, x0, #2')
x2 False
x0 False
x3 False
x1 False

('400d9c', 'cb010000', 'sub', None, 'x0, x0, x1')
x2 False
x0 False
x3 False
x1 False
x1 False

('400d98', 'd37be800', 'lsl', None, 'x0, x0, #5')
x2 False
x0 False
x3 False
x1 False
x1 False

('400d94', 'aa0103e0', 'mov', None, 'x0, x1')
x2 False
x1 False
x3 False
x1 False
x1 False

here
here is ldrsw True
here
here is ldrsw True
here
here is ldrsw True
('400d90', 'b9800fe1', 'ldrsw', None, 'x1, [sp, #12]')
x2 False
x1 False
x3 False
x1 False
x1 False

here
here is ldrsw True
('400d8c', 'b9800be3', 'ldrsw', None, 'x3, [sp, #8]')
x2 False
x1 False
x3 False
x1 False
x1 False

('400d88', '91010002', 'a

' ls_loop_info = lds_obj.loop_info\nlds_table = lds_obj.lsproc.ls_table\n\nfor i in lds_table:\n    if i.target_num == 1:\n        print("single",i.ins.tokens,"最终地址",i.final_addr,i.addr_offset,"是否找到",i.is_find,i.node.name) \n    else:\n        print("muti",i.ins.tokens,"最终地址",i.final_addr,i.addr_offset,"是否找到",i.is_find,i.node.name)\n\n\nfor k,v in ls_loop_info.items():\n    print(k,v) '

In [8]:
# from newCFG.heat_analysis import loop_heat
# test=loop_heat(tcfg,lsproc,r'C:\Users\13377\Desktop\hw-memory-master (5)\hw-memory-master\benchmarks\final_benchmark\spec_benchD.asm')
# test.do_it()


In [9]:

# TODO.
from cache_analysis.cache_risk_level import CacheRisk

cache_test = CacheRisk(tcfg, lsproc, r'D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmarkD.asm',
                       r'D:\workspace\hw-memory\cache_analysis\new_cache\input\cache_information.in')
cache_test.test()


ModuleNotFoundError: No module named 'cache_analysis.read_segment'

In [ ]:

# from newCFG.rw_condition import RWProc, RWType
# from newCFG.rw_condition_out import RWOut_Proc



# rwout = RWOut_Proc(tcfg_nodes,segreader,rwproc.rw_table,tcfg.loops)




